# Miscellaneous problems 
## The knapsack problem 
The knapsack problem is an optimization problem, finding the best use of limited resources given a finite set of usage options.

If a thief could take any amount of any item, he could simply divide each item’s value by its weight to figure out the most valuable items for the available capacity. But to make the scenario more realistic, let’s say that the thief cannot take half of an item (such as 2.5 televisions). Instead, we will come up with a way to solve the 0/1 variant of the problem, so-called because it enforces another rule: The thief may only take one or none of each item. 

In [1]:
from typing import NamedTuple, List

class Item(NamedTuple):
    name: str
    weight: int
    value: float

    def __repr__(self) -> str:
        return f"{self.name}: (weight: {self.weight}, value: {self.value})"

If we tried to solve this problem using a brute-force approach, we would look at every combination of items available to be put in the knapsack. For the mathematically inclined, this is known as a powerset, and a powerset of a set (in our case, the set of items) has 2^N different possible subsets, where N is the number of items. Therefore, we would need to analyze 2^N combinations (O(2^N)). This is okay for a small number of items, but it is untenable for a large number. Any approach that solves a problem using an exponential number of steps is an approach we want to avoid.

Instead, we will use a technique known as dynamic programming, which is similar in concept to memoization. Instead of solving a problem outright with a brute-force approach, in dynamic programming one solves subproblems that make up the larger problem, stores those results, and utilizes those stored results to solve the larger problem. As long as the capacity of the knapsack is considered in discrete steps, the problem can be solved with dynamic programming. 

All along the way we will fill in a table that tells us the best possible solution for each combination of items and capacity. Our function will first fill in the table and then figure out the solution based on the table.

In [2]:
def knapsack(items: List[Item], max_capacity: int) -> List[Item]:
    # build up dynamic programming table
    table: List[List[float]] = [[0.0 for _ in range(max_capacity + 1)] for _ in range(len(items) + 1)]
    for i, item in enumerate(items):
        for capacity in range(1, max_capacity + 1):
            previous_items_value: float = table[i][capacity]
            # item fits in knapsack
            if capacity >= item.weight: 
                value_freeing_weight_for_item: float = table[i][capacity - item.weight]
                # only take if more valuable than previous item
                table[i + 1][capacity] = max(value_freeing_weight_for_item + item.value, previous_items_value)
            else: 
                # no room for this item
                table[i + 1][capacity] = previous_items_value
                
    # figure out solution from table
    solution: List[Item] = []
    capacity = max_capacity
    for i in range(len(items), 0, -1): # work backwards
        # was this item used?
        if table[i - 1][capacity] != table[i][capacity]:
            solution.append(items[i - 1])
            # if the item was used, remove its weight
            capacity -= items[i - 1].weight
    return solution

| Item                    | 0 lb. | 1 lb. | 2 lb. | 3 lb. |
|-------------------------|-------|-------|-------|-------|
| Matches(1 lb., \$5)     |     0 |     5 |     5 |     5 |
| Flashlight(2 lb., \$10) |     0 |     5 |    10 |    15 |
| Book(1lb., \$15)        |     0 |    15 |    20 |    25 |

In [3]:
items: List[Item] = [Item("matches", 1, 5),
                     Item("flashlight", 2, 10),
                     Item("book", 1, 15)]
print(knapsack(items, 3))

[book: (weight: 1, value: 15), flashlight: (weight: 2, value: 10)]


In [4]:
items: List[Item] = [Item("television", 50, 500),
                     Item("candlesticks", 2, 300),
                     Item("stereo", 35, 400),
                     Item("laptop", 3, 1000),
                     Item("food", 15, 50),
                     Item("clothing", 20, 800),
                     Item("jewelry", 1, 4000),
                     Item("books", 100, 300),
                     Item("printer", 18, 30),
                     Item("refrigerator", 200, 700),
                     Item("painting", 10, 1000)]
print(knapsack(items, 75))

[painting: (weight: 10, value: 1000), jewelry: (weight: 1, value: 4000), clothing: (weight: 20, value: 800), laptop: (weight: 3, value: 1000), stereo: (weight: 35, value: 400), candlesticks: (weight: 2, value: 300)]


## The Traveling Salesman Problem 
The Traveling Salesman Problem is one of the most classic and talked-about problems in all of computing. A salesman must visit all of the cities on a map exactly once, returning to his start city at the end of the journey. There is a direct connection from every city to every other city, and the salesman may visit the cities in any order. What is the shortest path for the salesman? 

The problem can be thought of as a graph problem, with the cities being the vertices and the connections between them being the edges. Your first instinct might be to find the minimum spanning tree, as described in chapter 4. Unfortunately, the solution to the Traveling Salesman Problem is not so simple. The minimum spanning tree is the shortest way to connect all of the cities, but it does not provide the shortest path for visiting all of them exactly once. 

Although the problem, as posed, appears fairly simple, there is no algorithm that can solve it quickly for an arbitrary number of cities. 

### The naive approach
The naive approach to the problem is simply to try every possible combination of cities. Attempting the naive approach will illustrate the difficulty of the problem and this approach’s unsuitability for brute-force attempts at larger scales. 

In our version of the Traveling Salesman Problem, the salesman is interested in visiting five of the major cities of Vermont. 

We will need to codify both the cities and the distances between them for our problem. To make the distances between cities easy to look up, we will use a dictionary of dictionaries, with the outer set of keys representing the first of a pair and the inner set of keys representing the second. This will be the type Dict[str, Dict[str, int]].

In [5]:
from typing import Dict, List, Iterable, Tuple
from itertools import permutations

vt_distances: Dict[str, Dict[str, int]] = {
    "Rutland":
        {"Burlington": 67,
         "White River Junction": 46,
         "Bennington": 55,
         "Brattleboro": 75},
    "Burlington":
        {"Rutland": 67,
         "White River Junction": 91,
         "Bennington": 122,
         "Brattleboro": 153},
    "White River Junction":
        {"Rutland": 46,
         "Burlington": 91,
         "Bennington": 98,
         "Brattleboro": 65},
    "Bennington":
        {"Rutland": 55,
         "Burlington": 122,
         "White River Junction": 98,
         "Brattleboro": 40},
    "Brattleboro":
        {"Rutland": 75,
         "Burlington": 153,
         "White River Junction": 65,
         "Bennington": 40}
}

The naive approach to solving the Traveling Salesman Problem requires generating every possible permutation of the cities. There are many permutation-generation algorithms; they are simple enough to ideate that you could almost certainly come up with one on your own. 

Luckily, there is no need to reinvent the wheel by writing a permutation-generation algorithm, because the Python standard library has a permutations() function in its itertools module. In the following code snippet, we generate all of the permutations of the Vermont cities that our travelling salesman would need to visit. Because there are five cities, this is 5! (5 factorial), or 120 permutations.

Recall that in the Traveling Salesman Problem, the salesman must return, at the end, to the same city that he started in. We can easily add the first city in a permutation to the end of a permutation using a list comprehension. 

In [6]:
vt_cities: Iterable[str] = vt_distances.keys()
city_permutations: Iterable[Tuple[str, ...]] = permutations(vt_cities) 
tsp_paths: List[Tuple[str, ...]] = [c + (c[0],) for c in city_permutations]

We are now ready to try testing the paths we have permuted. A brute-force search approach painstakingly looks at every path in a list of paths and uses the distance between two cities lookup table (vt_distances) to calculate each path’s total distance. It prints both the shortest path and that path’s total distance. 

In [7]:
best_path: Tuple[str, ...]
min_distance: int = 99999999999 # arbitrarily high number
for path in tsp_paths:
    distance: int = 0
    last: str = path[0]
    for next in path[1:]:
        distance += vt_distances[last][next]
        last = next
    if distance < min_distance:
        min_distance = distance
        best_path = path
print(f"The shortest path is {best_path} in {min_distance} miles.")

The shortest path is ('Rutland', 'Burlington', 'White River Junction', 'Brattleboro', 'Bennington', 'Rutland') in 318 miles.


## Phone number mnemonics
Telephones included letters on each of the keys on their number pads. The reason for these letters was to provide easy mnemonics by which to remember phone numbers.

In [8]:
from typing import Dict, Tuple, Iterable, List
from itertools import product

phone_mapping: Dict[str, Tuple[str, ...]] = {"1": ("1",),
                                             "2": ("a", "b", "c"),
                                             "3": ("d", "e", "f"),
                                             "4": ("g", "h", "i"),
                                             "5": ("j", "k", "l"),
                                             "6": ("m", "n", "o"),
                                             "7": ("p", "q", "r", "s"),
                                             "8": ("t", "u", "v"),
                                             "9": ("w", "x", "y", "z"),
                                             "0": ("0",)}

The next function combines all of those possibilities for each numeral into a list of possible mnemonics for a given phone number. It does this by creating a list of tuples of potential letters for each digit in the phone number and then combining them through the Cartesian product function product() from itertools. Note the use of the unpack (*) operator to use the tuples in letter_tuples as the arguments for product(). 

In [9]:
def possible_mnemonics(phone_number: str) -> Iterable[Tuple[str, ...]]:
    letter_tuples: List[Tuple[str, ...]] = []
    for digit in phone_number:
        letter_tuples.append(phone_mapping.get(digit, (digit,)))
    return product(*letter_tuples)

In [10]:
phone_number: str = input("Enter a phone number:")
print("Here are the potential mnemonics:")
for mnemonic in possible_mnemonics(phone_number):
    print("".join(mnemonic))

Enter a phone number:3345
Here are the potential mnemonics:
ddgj
ddgk
ddgl
ddhj
ddhk
ddhl
ddij
ddik
ddil
degj
degk
degl
dehj
dehk
dehl
deij
deik
deil
dfgj
dfgk
dfgl
dfhj
dfhk
dfhl
dfij
dfik
dfil
edgj
edgk
edgl
edhj
edhk
edhl
edij
edik
edil
eegj
eegk
eegl
eehj
eehk
eehl
eeij
eeik
eeil
efgj
efgk
efgl
efhj
efhk
efhl
efij
efik
efil
fdgj
fdgk
fdgl
fdhj
fdhk
fdhl
fdij
fdik
fdil
fegj
fegk
fegl
fehj
fehk
fehl
feij
feik
feil
ffgj
ffgk
ffgl
ffhj
ffhk
ffhl
ffij
ffik
ffil
